In [31]:
import numpy as np #type: ignore
import pandas as pd #type: ignore

import tensorflow as tf #type: ignore
from tensorflow.keras import layers, Model #type: ignore
from tensorflow.keras.layers import (Conv2D,MaxPooling1D, MaxPooling2D, Flatten,MaxPool1D, Dense, 
                                     LSTM, Bidirectional, Input, RepeatVector, 
                                     Concatenate, Dropout, GlobalAveragePooling2D, 
                                     Conv1D, BatchNormalization, Attention) #type: ignore

from sklearn.model_selection import StratifiedKFold #type: ignore
from sklearn.utils.class_weight import compute_class_weight #type: ignore
from functools import partial #type: ignore
import math #type: ignore
from tqdm import tqdm #type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import EarlyStopping #  type: ignore
from imblearn.combine import SMOTETomek       # type: ignore



In [32]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
train_df['event_id'] = train_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
train_df['event_idx'] = train_df.groupby('event_id', sort=False).ngroup()
test_df['event_id'] = test_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
test_df['event_idx'] = test_df.groupby('event_id', sort=False).ngroup()

train_df['event_t'] = train_df.groupby('event_id').cumcount()
test_df['event_t'] = test_df.groupby('event_id').cumcount()

print(train_df.head())
print(test_df.head())

          event_id  precipitation  label  event_idx  event_t
0  id_spictby0jfsb       0.000000      0          0        0
1  id_spictby0jfsb       0.095438      0          0        1
2  id_spictby0jfsb       1.949560      0          0        2
3  id_spictby0jfsb       3.232160      0          0        3
4  id_spictby0jfsb       0.000000      0          0        4
          event_id  precipitation  event_idx  event_t
0  id_j7b6sokflo4k        0.00000          0        0
1  id_j7b6sokflo4k        3.01864          0        1
2  id_j7b6sokflo4k        0.00000          0        2
3  id_j7b6sokflo4k       16.61520          0        3
4  id_j7b6sokflo4k        2.56706          0        4


In [7]:
# Constants for image preprocessing
BAND_NAMES = ('B2', 'B3', 'B4', 'B8', 'B11', 'slope')
H, W, NUM_CHANNELS = IMG_DIM = (128, 128, len(BAND_NAMES))
_MAX_INT = np.iinfo(np.uint16).max

def decode_slope(x):
    return (x / _MAX_INT * (math.pi / 2.0)).astype(np.float32)

def normalize(x, mean, std):
    return (x - mean) / std

rough_S2_normalize = partial(normalize, mean=1250, std=500)


In [8]:
def preprocess_image(x):
    return np.concatenate([
        rough_S2_normalize(x[..., :-1].astype(np.float32)),
        decode_slope(x[..., -1:]),
    ], axis=-1, dtype=np.float32)

def preprocess_precipitation(x):
    return (x - np.mean(x)) / np.std(x)  # Standardize precipitation data


In [9]:
# Load composite images
with np.load('composite_images.npz') as data:
    composite_images = {event_id: preprocess_image(data[event_id]) for event_id in data.keys()}

# Placeholder for missing images
def get_image(event_id):
    if event_id in composite_images:
        return composite_images[event_id]
    return np.zeros((128, 128, 6), dtype=np.float32)  # Zero image for missing data


In [10]:
# Preprocess time series and images
def preprocess_data_and_images(data_df):
    event_ids = data_df['event_id'].unique()
    timeseries, labels, images = [], [], []

    for event_id in tqdm(event_ids, desc='Processing data'):
        event_data = data_df[data_df['event_id'] == event_id]
        timeseries.append(preprocess_precipitation(event_data['precipitation'].values))
        if 'label' in event_data.columns:
            labels.append(event_data['label'].values)
        images.append(get_image(event_id))
    
    return np.array(timeseries), np.array(labels) if labels else None, np.stack(images, axis=0)


In [11]:
train_timeseries, train_labels, train_images = preprocess_data_and_images(train_df)
test_timeseries, _, test_images = preprocess_data_and_images(test_df)

Processing data: 100%|██████████| 224/224 [00:04<00:00, 47.67it/s]


In [33]:

# Define a deeper CNN for image feature extraction
def build_image_encoder():
    image_input = Input(shape=(128, 128, 6), name='image_input')
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.2)(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    encoded_image = Dense(128, activation='relu')(x)
    return Model(image_input, encoded_image, name='Image_Encoder')

# Define time series processing with CNN + LSTM
def build_time_series_model():
    precip_input = Input(shape=(730,), name='precip_input')
    expanded_input = layers.Reshape((730, 1))(precip_input)
    x = Conv1D(64, 3, activation='relu', padding='same')(expanded_input)
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    attn = Attention()([x, x])  # Attention mechanism
    x = Dense(64, activation='relu')(attn)
    return Model(precip_input, x, name='TimeSeries_Model')



In [34]:
# Merge models
image_encoder = build_image_encoder()
time_series_model = build_time_series_model()

def build_flood_prediction_model():
    encoded_image = image_encoder.output
    repeated_image_vector = RepeatVector(730)(encoded_image)
    
    encoded_timeseries = time_series_model.output
    concatenated = Concatenate(axis=-1)([repeated_image_vector, encoded_timeseries])
    x = Dense(64, activation='relu')(concatenated)
    x = Dropout(0.2)(x)
    day_probabilities = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[image_encoder.input, time_series_model.input], outputs=day_probabilities)
    return model

model = build_flood_prediction_model()


In [36]:
# Define Focal Loss
def focal_loss(alpha=0.25, gamma=2.):
    def loss(y_true, y_pred):
        epsilon = 1e-8
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        ce = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        return tf.reduce_mean(weight * ce)
    return loss

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=focal_loss(), metrics=['accuracy'])


In [37]:
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels.argmax(axis=1)), y=train_labels.argmax(axis=1))
class_weight_dict = {i: w for i, w in enumerate(class_weights)}

In [43]:
# Train model
model.fit(
    [train_images, train_timeseries], train_labels,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 4.5842e-04 - loss: 7.5771e-10 - val_accuracy: 0.0014 - val_loss: 1.3459e-07
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 4.8492e-04 - loss: 3.4139e-09 - val_accuracy: 0.0014 - val_loss: 1.3051e-07
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 4.7318e-04 - loss: 8.7299e-10 - val_accuracy: 0.0014 - val_loss: 1.2823e-07
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 4.5334e-04 - loss: 9.9308e-10 - val_accuracy: 0.0014 - val_loss: 1.2921e-07
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 4.7379e-04 - loss: 7.2722e-10 - val_accuracy: 0.0014 - val_loss: 1.2784e-07
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 4.6740e-04 - loss: 9.2286e-10 - val_accuracy: 0.0014 - val_loss: 1.2545e-07
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 4.2579e-04 - loss: 8.9500e-10 - val_accuracy: 0.0014 - val_loss: 1.2348e-07
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━

In [44]:
predictions = model.predict([test_images, test_timeseries])

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 662ms/step


In [45]:
# Save submission
sample_submission = pd.read_csv('SampleSubmission (2).csv')
sample_submission['label'] = predictions.flatten()
sample_submission.to_csv("submission_optimized2.csv", index=False)


In [46]:
sample_submission.head()

,event_id,label
0,id_j7b6sokflo4k_X_0,0.981471
1,id_j7b6sokflo4k_X_1,0.981485
2,id_j7b6sokflo4k_X_2,0.981506
3,id_j7b6sokflo4k_X_3,0.981526
4,id_j7b6sokflo4k_X_4,0.981531
